# Neural integration for constitutive equations 

### Benchmark #2: porous, elasto-plastic material

Authors: Filippo Masi, Itai Einav

### 1. Import libraries

In [1]:
import pickle
import numpy as np
import torch
from scipy.optimize import root
from torchdiffeq import odeint
from nice_module import NICE, EarlyStopping, shuffle, slice_data, GetParams

np.random.seed(6)
torch.manual_seed(6)

# plotting
import matplotlib.pyplot as plt
import matplotlib
plt.style.use('classic')
plt.rcParams.update({"axes.grid" : False, "grid.color": 'black', "grid.alpha":0.4})
font = {'size'   : 10}
matplotlib.rc('font', **font)
plt.rcParams['axes.facecolor']='none'
plt.rcParams['savefig.facecolor']='none'
plt.rcParams['figure.facecolor']='none'
plt.rcParams["figure.figsize"] = (2.5,2)
plt.tight_layout(pad=2.5, w_pad=3.5, h_pad=3.5)
colorb = (0.2,0.4,0.7)

<Figure size 200x160 with 0 Axes>

#### 1.1 Set hyperparameters

In [2]:
verbose_frequency = 10 # frequency in epochs for printing loss at training
step_size = 1 # to reproduce training process set = 20, else 1 (for fast training)
device = torch.device('cuda:' + str(gpu) if torch.cuda.is_available() else 'cpu')
corrupted_training_data = False # boolean for adding normally distributed noise to training data set
delta = 0. # if corrupted_training_data = True, set noise amplitude (percentage)

### 2. Import and prepare data sets

In [3]:
file = './dataset/benchmark2_data_training'
with open(file, 'rb') as f_obj:
    data = pickle.load(f_obj)

strain_t,strain_tdt,r_t,r_tdt,z_t,z_tdt,stress_t,dt,n_reset = data
batch_time = n_reset
data_size = n_reset
dim = 2
prm_dt = 1/data_size

140


#### 3.1 Reshape 


In [12]:
dstrain = strain_tdt - strain_t
dr = r_tdt - r_t
dz = z_tdt - z_t
dstrain/=prm_dt
dr/=prm_dt
dz/=prm_dt

strain_t = np.reshape(strain_t,(batch_time,-1,dim),order='F')
strain_tdt = np.reshape(strain_tdt,(batch_time,-1,dim),order='F')
dstrain = np.reshape(dstrain,(batch_time,-1,dim),order='F')
r_t = np.reshape(r_t,(batch_time,-1,1),order='F')
dr = np.reshape(dr,(batch_time,-1,1),order='F')
z_t = np.reshape(z_t,(batch_time,-1,1),order='F')
dz = np.reshape(dz,(batch_time,-1,1),order='F')
stress_t = np.reshape(stress_t,(batch_time,-1,dim),order='F')

data_size = strain_t.shape[0]
number_IC = strain_t.shape[1]

#### 3.2 Split data in training, validation, and test sets 

In [13]:
train_percentage = .65

train = int(round(number_IC * train_percentage))
val = int(round(number_IC * 0.5 *(1.- train_percentage)))
test = val

print("Number of samples: ", number_IC)
print("Training samples : ", train)
print("Validation samples : ", val)
print("Test samples : ", test)
print("Total : ", test + val + train)

Number of samples:  52
Training samples :  34
Validation samples :  9
Test samples :  9
Total :  52


In [14]:
n = np.arange(0,number_IC,1)
rnd = np.arange(len(n))
np.random.shuffle(rnd[:-4])
n = n[rnd]

ntrain = n[:train-4]
ntrain = np.hstack((ntrain,n[-4:]))
cut = len(ntrain)
nval = n[train:train+val]
ntrainval = np.hstack((ntrain,nval))
# ntest = np.arange(0,number_IC)
ntest = n[train+val:]


rnd = np.arange(len(ntrainval))
ntrain = rnd[:train]
nval = rnd[val:]

In [15]:
strain_t_tv,strain_t_test = slice_data(strain_t,ntrainval,ntest)
dstrain_tv,dstrain_test = slice_data(dstrain,ntrainval,ntest)
stress_t_tv,stress_t_test = slice_data(stress_t,ntrainval,ntest)
r_t_tv,r_t_test = slice_data(r_t,ntrainval,ntest)
dr_tv,dr_test = slice_data(dr,ntrainval,ntest)
z_t_tv,z_t_test = slice_data(z_t,ntrainval,ntest)
dz_tv,dz_test = slice_data(dz,ntrainval,ntest)

In [16]:
prm_e = GetParams(strain_t_tv).to(device)
prm_de = GetParams(dstrain_tv).to(device)
prm_s = GetParams(stress_t_tv).to(device)
prm_r = GetParams(r_t_tv).to(device)
prm_dr = GetParams(dr_tv).to(device)
prm_z = GetParams(z_t_tv).to(device)
prm_dz = GetParams(dz_tv).to(device)

In [17]:
if corrupted_training_data:
    # Stress noise vector
    noise = delta/100 * np.random.normal(0,1,((data_size+1,number_IC,dim)))
    # Corrupt stress with noise
    noise_stress_t = stress_t_tv.copy()
    noise_stress_t[:,:,0] = np.mean(stress_t_tv[:,:,0])*noise[:-1,:,0]
    noise_stress_t[:,:,1] = np.mean(stress_t_tv[:,:,1])*noise[:-1,:,1]
    noise_stress_t[0] *=0.0
    stress_t_tv += noise_stress_t
    # Dissipative state variable noise vector
    noise = delta/100 * np.random.normal(0,1,((data_size+1,number_IC,1)))
    # Corrupt state variable with noise
    noise_svars_z_t = z_t_tv.copy()
    noise_svars_z_tdt = z_tdt_tv.copy()
    noise_svars_z_t = np.mean(z_t_tv)*noise[:-1,:]
    noise_svars_z_tdt = np.mean(z_tdt_tv)*noise[1:,:]
    noise_svars_z_t[0] *=0.0
    z_t_tv[:,:] += noise_svars_z_t
    z_tdt_tv[:,:] += noise_svars_z_tdt
    # Evaluate corrupted rate
    dz_tv = (z_tdt_tv - z_t_tv)/prm_dt

### 4. Neural integration for constitutive equations

#### 4.1 Constructu neural net and set integration scheme

In [18]:
dtype=torch.float64
hidden_num = 0
NNf_params = [2*dim+2,dim+1,[6*(2*dim+3),6*(2*dim+3),6*(2*dim+3),],'gelu']
NNu_params = [dim+2,1,[2**6,2**6],'softplus']
norm_params = [prm_e,prm_de,prm_r,prm_z,prm_dz,prm_s,prm_dt]
nsvars = 3
NICE_network = NICE(NNf_params,NNu_params,hidden_num,len(ntrainval),norm_params,dim,nsvars,dtype).to(device)
NICE_network.to(torch.double)

NICE(
  (NeuralNetEvolution): Sequential(
    (0): Linear(in_features=6, out_features=42, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=42, out_features=42, bias=True)
    (3): GELU(approximate='none')
    (4): Linear(in_features=42, out_features=42, bias=True)
    (5): GELU(approximate='none')
    (6): Linear(in_features=42, out_features=3, bias=True)
  )
  (NeuralNetEnergy): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): Softplus(beta=1, threshold=20)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Softplus(beta=1, threshold=20)
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
  (relu): ReLU()
)

In [19]:
prm_dt = 1/data_size
t = torch.arange(0,1.0,prm_dt)
stress_tv = torch.from_numpy(np.float64(stress_t_tv)).to(device)
dstrain_tv = torch.from_numpy(np.float64(dstrain_tv)).to(device)
stress_test = torch.from_numpy(np.float64(stress_t_test)).to(device)
dstrain_test = torch.from_numpy(np.float64(dstrain_test)).to(device)

svars_tv = torch.cat((torch.from_numpy(np.float64(r_t_tv)),torch.from_numpy(np.float64(z_t_tv)),),-1).to(device)
svars_test = torch.cat((torch.from_numpy(np.float64(r_t_test)),torch.from_numpy(np.float64(z_t_test)),),-1).to(device)


In [20]:
NICE_network.solver = "midpoint"
NICE_network.scheme = "forward"
NICE_network.step_size = prm_dt/step_size
NICE_network.init_interp(dstrain_tv,t)
NICE_network.inference = False

#### 4.2 Training

In [21]:
# Optimizer, loss function, and hyperparameters
learningRate = 1e-2
optimizer =  torch.optim.Adam(NICE_network.parameters(),lr=learningRate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer,gamma=0.999)
w_reg = 1.e-5 # l2 penalty weight 
Nepochs = 1000000 # number of epochs
MSE = torch.nn.MSELoss()

# Early stopping criterion
checkpoint_path = 'checkpoint_benchmark2_dense.pt'
early_stopping = EarlyStopping(patience=10000, delta=1.e-9, verbose=False, path=checkpoint_path)

# List for storing training and validation loss
training_loss_hist = []
validation_loss_value_hist = []

In [ ]:
par_svars0 = svars_tv[0]
par_svars0.requires_grad=True

for epoch in range(1,Nepochs):
    optimizer.zero_grad()

    ueps_e_0 = NICE_network.DeNormalize(NICE_network.e0,NICE_network.prm_ee)
    usvars0 = torch.cat((ueps_e_0,par_svars0),-1)
    stress0 = NICE_network.stress([ueps_e_0,par_svars0[:,:1],par_svars0[:,1:]])
    
    pred = NICE_network.integrate(dstrain_tv, usvars0, t, np.hstack((ntrain,nval)))
    pred_svars,pred_stress,pred_diss = pred
    
    training_loss_stress = MSE(NICE_network.Normalize(pred_stress[:,nval],prm_s),
                                    NICE_network.Normalize(stress_tv[:,nval],prm_s))
    training_loss_r0 = MSE(NICE_network.Normalize(stress0,prm_s),
                                NICE_network.Normalize(stress_tv[0],prm_s))
    training_loss_svars = (MSE(NICE_network.Normalize(pred_svars[:,ntrain,-1],prm_z),
                               NICE_network.Normalize(svars_tv[:,ntrain,-1],prm_z)))

    norm_d = torch.max(torch.abs(pred_diss)).detach()
    training_loss_dissipation = MSE(NICE_network.relu(-pred_diss[:,ntrain])/norm_d, pred_diss[:,ntrain].detach()*0)

    l_reg = torch.tensor(0.,requires_grad=True)
    for name, param in NICE_network.named_parameters():
        if 'weight' in name: l_reg = l_reg + pow(param,2).sum()

    training_loss = (torch.mean(training_loss_stress)
                     + torch.mean(training_loss_r0)
                     + torch.mean(training_loss_svars)
                     + torch.mean(training_loss_dissipation)
                     + w_reg*l_reg
                    )
    training_loss.backward()
    optimizer.step()
        
    if scheduler.get_last_lr()[0]>1.e-4: scheduler.step()
        
    validation_loss_stress = MSE(NICE_network.Normalize(pred_stress[:,nval],prm_s),
                                 NICE_network.Normalize(stress_tv[:,nval],prm_s))
    validation_loss_svars = (MSE(NICE_network.Normalize(pred_svars[:,nval,-1],prm_z),
                                 NICE_network.Normalize(svars_tv[:,nval,-1],prm_z)))
    validation_loss_dissipation = MSE(NICE_network.relu(-pred_diss[:,nval])/norm_d,pred_diss[:,nval].detach()*0)
    validation_loss = (torch.mean(validation_loss_stress)
                       + torch.mean(validation_loss_svars)
                       + torch.mean(validation_loss_dissipation))    
    
    training_loss_value = training_loss.item()
    validation_loss_value = validation_loss.item()
    training_loss_hist.append(training_loss_value)
    validation_loss_value_hist.append(validation_loss_value)
    
    # Print loss
    if not epoch % verbose_frequency:
        print(f"Epoch: {epoch}"
          + f" | training loss: {training_loss_value:.4e}"
          + f" | validation loss: {validation_loss_value:.4e}")

    # Check early_stopping criterion
    early_stopping(validation_loss_value, NICE_network)
    if early_stopping.early_stop :
        print("Early stopping")
        NICE_network.load_state_dict(torch.load(checkpoint_path))
        break   

Epoch: 10 | training loss: 1.9945e+00 | validation loss: 1.6620e+00
Epoch: 20 | training loss: 9.4394e-01 | validation loss: 8.1216e-01
Epoch: 30 | training loss: 5.5585e-01 | validation loss: 3.7120e-01
Epoch: 40 | training loss: 3.7634e-01 | validation loss: 2.9455e-01
Epoch: 50 | training loss: 2.2203e-01 | validation loss: 1.7295e-01
Epoch: 60 | training loss: 1.0998e-01 | validation loss: 8.0650e-02
Epoch: 70 | training loss: 5.6262e-02 | validation loss: 3.9742e-02
Epoch: 80 | training loss: 2.9969e-02 | validation loss: 2.1494e-02
Epoch: 90 | training loss: 1.6989e-02 | validation loss: 1.3086e-02
Epoch: 100 | training loss: 1.1549e-02 | validation loss: 9.4721e-03
Epoch: 110 | training loss: 9.2760e-03 | validation loss: 7.7135e-03
Epoch: 120 | training loss: 7.8282e-03 | validation loss: 6.6760e-03
Epoch: 130 | training loss: 6.8902e-03 | validation loss: 5.7392e-03
Epoch: 140 | training loss: 6.2779e-03 | validation loss: 5.0719e-03
Epoch: 150 | training loss: 5.8175e-03 | va

Epoch: 1200 | training loss: 2.0907e-03 | validation loss: 6.3900e-04
Epoch: 1210 | training loss: 2.0860e-03 | validation loss: 6.3458e-04
Epoch: 1220 | training loss: 2.0813e-03 | validation loss: 6.3024e-04
Epoch: 1230 | training loss: 2.0767e-03 | validation loss: 6.2598e-04
Epoch: 1240 | training loss: 2.0722e-03 | validation loss: 6.2180e-04
Epoch: 1250 | training loss: 2.0677e-03 | validation loss: 6.1769e-04
Epoch: 1260 | training loss: 2.0633e-03 | validation loss: 6.1367e-04
Epoch: 1270 | training loss: 2.0590e-03 | validation loss: 6.0971e-04
Epoch: 1280 | training loss: 2.0548e-03 | validation loss: 6.0583e-04
Epoch: 1290 | training loss: 2.0506e-03 | validation loss: 6.0201e-04
Epoch: 1300 | training loss: 2.0464e-03 | validation loss: 5.9826e-04
Epoch: 1310 | training loss: 2.0424e-03 | validation loss: 5.9456e-04
Epoch: 1320 | training loss: 2.0383e-03 | validation loss: 5.9079e-04
Epoch: 1330 | training loss: 2.0345e-03 | validation loss: 5.8587e-04
Epoch: 1340 | traini

Epoch: 2380 | training loss: 1.7988e-03 | validation loss: 4.0194e-04
Epoch: 2390 | training loss: 1.7975e-03 | validation loss: 4.0106e-04
Epoch: 2400 | training loss: 1.7962e-03 | validation loss: 4.0019e-04
Epoch: 2410 | training loss: 1.7949e-03 | validation loss: 3.9933e-04
Epoch: 2420 | training loss: 1.7936e-03 | validation loss: 3.9848e-04
Epoch: 2430 | training loss: 1.7923e-03 | validation loss: 3.9764e-04
Epoch: 2440 | training loss: 1.7911e-03 | validation loss: 3.9681e-04
Epoch: 2450 | training loss: 1.7898e-03 | validation loss: 3.9598e-04
Epoch: 2460 | training loss: 1.7886e-03 | validation loss: 3.9517e-04
Epoch: 2470 | training loss: 1.7874e-03 | validation loss: 3.9436e-04
Epoch: 2480 | training loss: 1.7862e-03 | validation loss: 3.9355e-04
Epoch: 2490 | training loss: 1.7850e-03 | validation loss: 3.9276e-04
Epoch: 2500 | training loss: 1.7838e-03 | validation loss: 3.9197e-04
Epoch: 2510 | training loss: 1.7826e-03 | validation loss: 3.9120e-04
Epoch: 2520 | traini

Epoch: 3560 | training loss: 1.6958e-03 | validation loss: 3.3800e-04
Epoch: 3570 | training loss: 1.6952e-03 | validation loss: 3.3768e-04
Epoch: 3580 | training loss: 1.6947e-03 | validation loss: 3.3736e-04
Epoch: 3590 | training loss: 1.6941e-03 | validation loss: 3.3704e-04
Epoch: 3600 | training loss: 1.6936e-03 | validation loss: 3.3672e-04
Epoch: 3610 | training loss: 1.6930e-03 | validation loss: 3.3641e-04
Epoch: 3620 | training loss: 1.6925e-03 | validation loss: 3.3610e-04
Epoch: 3630 | training loss: 1.6919e-03 | validation loss: 3.3579e-04
Epoch: 3640 | training loss: 1.6914e-03 | validation loss: 3.3548e-04
Epoch: 3650 | training loss: 1.6909e-03 | validation loss: 3.3518e-04
Epoch: 3660 | training loss: 1.6903e-03 | validation loss: 3.3488e-04
Epoch: 3670 | training loss: 1.6898e-03 | validation loss: 3.3458e-04
Epoch: 3680 | training loss: 1.6893e-03 | validation loss: 3.3428e-04
Epoch: 3690 | training loss: 1.6888e-03 | validation loss: 3.3398e-04
Epoch: 3700 | traini

Epoch: 4740 | training loss: 1.6481e-03 | validation loss: 3.1169e-04
Epoch: 4750 | training loss: 1.6478e-03 | validation loss: 3.1152e-04
Epoch: 4760 | training loss: 1.6475e-03 | validation loss: 3.1135e-04
Epoch: 4770 | training loss: 1.6472e-03 | validation loss: 3.1118e-04
Epoch: 4780 | training loss: 1.6469e-03 | validation loss: 3.1101e-04
Epoch: 4790 | training loss: 1.6465e-03 | validation loss: 3.1083e-04
Epoch: 4800 | training loss: 1.6462e-03 | validation loss: 3.1066e-04
Epoch: 4810 | training loss: 1.6459e-03 | validation loss: 3.1049e-04
Epoch: 4820 | training loss: 1.6456e-03 | validation loss: 3.1032e-04
Epoch: 4830 | training loss: 1.6452e-03 | validation loss: 3.1014e-04
Epoch: 4840 | training loss: 1.6449e-03 | validation loss: 3.0997e-04
Epoch: 4850 | training loss: 1.6446e-03 | validation loss: 3.0979e-04
Epoch: 4860 | training loss: 1.6442e-03 | validation loss: 3.0962e-04
Epoch: 4870 | training loss: 1.6439e-03 | validation loss: 3.0944e-04
Epoch: 4880 | traini

...

Epoch: 32670 | training loss: 7.6062e-04 | validation loss: 7.2384e-05
Epoch: 32680 | training loss: 7.6056e-04 | validation loss: 7.2377e-05
Epoch: 32690 | training loss: 7.6049e-04 | validation loss: 7.2371e-05
Epoch: 32700 | training loss: 7.6043e-04 | validation loss: 7.2364e-05
Epoch: 32710 | training loss: 7.6037e-04 | validation loss: 7.2357e-05
Epoch: 32720 | training loss: 7.6030e-04 | validation loss: 7.2351e-05
Epoch: 32730 | training loss: 7.6024e-04 | validation loss: 7.2344e-05
Epoch: 32740 | training loss: 7.6017e-04 | validation loss: 7.2337e-05
Epoch: 32750 | training loss: 7.6010e-04 | validation loss: 7.2331e-05
Epoch: 32760 | training loss: 7.6004e-04 | validation loss: 7.2324e-05
Epoch: 32770 | training loss: 7.5997e-04 | validation loss: 7.2317e-05
Epoch: 32780 | training loss: 7.5991e-04 | validation loss: 7.2311e-05
Epoch: 32790 | training loss: 7.5986e-04 | validation loss: 7.2334e-05
Epoch: 32800 | training loss: 7.7591e-04 | validation loss: 8.3605e-05
Epoch:

Epoch: 33830 | training loss: 7.5376e-04 | validation loss: 7.1773e-05
Epoch: 33840 | training loss: 7.5371e-04 | validation loss: 7.1768e-05
Epoch: 33850 | training loss: 7.5365e-04 | validation loss: 7.1763e-05
Epoch: 33860 | training loss: 7.5360e-04 | validation loss: 7.1758e-05
Epoch: 33870 | training loss: 7.5354e-04 | validation loss: 7.1752e-05
Epoch: 33880 | training loss: 7.5348e-04 | validation loss: 7.1747e-05
Epoch: 33890 | training loss: 7.5343e-04 | validation loss: 7.1742e-05
Epoch: 33900 | training loss: 7.5337e-04 | validation loss: 7.1736e-05
Epoch: 33910 | training loss: 7.5331e-04 | validation loss: 7.1731e-05
Epoch: 33920 | training loss: 7.5326e-04 | validation loss: 7.1726e-05
Epoch: 33930 | training loss: 7.5320e-04 | validation loss: 7.1720e-05
Epoch: 33940 | training loss: 7.5314e-04 | validation loss: 7.1715e-05
Epoch: 33950 | training loss: 7.5309e-04 | validation loss: 7.1709e-05
Epoch: 33960 | training loss: 7.5303e-04 | validation loss: 7.1703e-05
Epoch:

Epoch: 34990 | training loss: 7.4724e-04 | validation loss: 7.1216e-05
Epoch: 35000 | training loss: 7.4718e-04 | validation loss: 7.1211e-05
Epoch: 35010 | training loss: 7.4712e-04 | validation loss: 7.1205e-05
Epoch: 35020 | training loss: 7.4706e-04 | validation loss: 7.1202e-05
Epoch: 35030 | training loss: 7.4722e-04 | validation loss: 7.1402e-05
Epoch: 35040 | training loss: 8.0020e-04 | validation loss: 1.0815e-04
Epoch: 35050 | training loss: 7.5931e-04 | validation loss: 8.0297e-05
Epoch: 35060 | training loss: 7.4750e-04 | validation loss: 7.1547e-05
Epoch: 35070 | training loss: 7.4827e-04 | validation loss: 7.2042e-05
Epoch: 35080 | training loss: 7.4692e-04 | validation loss: 7.1367e-05
Epoch: 35090 | training loss: 7.4686e-04 | validation loss: 7.1342e-05
Epoch: 35100 | training loss: 7.4666e-04 | validation loss: 7.1179e-05
Epoch: 35110 | training loss: 7.4662e-04 | validation loss: 7.1178e-05
Epoch: 35120 | training loss: 7.4654e-04 | validation loss: 7.1173e-05
Epoch:

Epoch: 36150 | training loss: 7.4112e-04 | validation loss: 7.0726e-05
Epoch: 36160 | training loss: 7.4107e-04 | validation loss: 7.0721e-05
Epoch: 36170 | training loss: 7.4102e-04 | validation loss: 7.0716e-05
Epoch: 36180 | training loss: 7.4097e-04 | validation loss: 7.0712e-05
Epoch: 36190 | training loss: 7.4092e-04 | validation loss: 7.0707e-05
Epoch: 36200 | training loss: 7.4086e-04 | validation loss: 7.0702e-05
Epoch: 36210 | training loss: 7.4081e-04 | validation loss: 7.0697e-05
Epoch: 36220 | training loss: 7.4076e-04 | validation loss: 7.0692e-05
Epoch: 36230 | training loss: 7.4071e-04 | validation loss: 7.0688e-05
Epoch: 36240 | training loss: 7.4065e-04 | validation loss: 7.0683e-05
Epoch: 36250 | training loss: 7.4060e-04 | validation loss: 7.0678e-05
Epoch: 36260 | training loss: 7.4055e-04 | validation loss: 7.0673e-05
Epoch: 36270 | training loss: 7.4049e-04 | validation loss: 7.0668e-05
Epoch: 36280 | training loss: 7.4044e-04 | validation loss: 7.0663e-05
Epoch:

Epoch: 37310 | training loss: 7.6533e-04 | validation loss: 9.0087e-05
Epoch: 37320 | training loss: 7.3660e-04 | validation loss: 7.1169e-05
Epoch: 37330 | training loss: 7.3841e-04 | validation loss: 7.2715e-05
Epoch: 37340 | training loss: 7.3513e-04 | validation loss: 7.0261e-05
Epoch: 37350 | training loss: 7.3545e-04 | validation loss: 7.0429e-05
Epoch: 37360 | training loss: 7.3502e-04 | validation loss: 7.0259e-05
Epoch: 37370 | training loss: 7.3501e-04 | validation loss: 7.0285e-05
Epoch: 37380 | training loss: 7.3491e-04 | validation loss: 7.0223e-05
Epoch: 37390 | training loss: 7.3488e-04 | validation loss: 7.0218e-05
Epoch: 37400 | training loss: 7.3482e-04 | validation loss: 7.0216e-05
Epoch: 37410 | training loss: 7.3478e-04 | validation loss: 7.0216e-05
Epoch: 37420 | training loss: 7.3473e-04 | validation loss: 7.0213e-05
Epoch: 37430 | training loss: 7.3469e-04 | validation loss: 7.0207e-05
Epoch: 37440 | training loss: 7.3464e-04 | validation loss: 7.0203e-05
Epoch:

In [ ]:
1/0

In [ ]:
checkpoint_path = './checkpoint_benchmark2_dense.pt'
NICE_network.load_state_dict(torch.load(checkpoint_path))

In [ ]:
t = prm_dt*torch.linspace(0., data_size-1, (data_size)).to(device)
svars_tv.requires_grad=True
sol = root(NICE_network.find_elastic_strain,
           args=([svars_tv[0,:,:1].reshape(-1,1),
                 svars_tv[0,:,-1:].reshape(-1,1)],
                 stress_tv[0].reshape(-1,2)),
           x0=np.zeros((len(ntrainval),2)),
           tol=1e-12)
eps_e_0 = torch.from_numpy(sol.x.reshape(-1,2))
ueps_e_0 = NICE_network.DeNormalize(eps_e_0,NICE_network.prm_ee)
usvars = torch.cat((ueps_e_0,svars_tv[0]),-1)

pred_svars,pred_stress,pred_diss = NICE_network.integrate(dstrain_tv[:],usvars,t,np.hstack((ntrain,nval)))
# Evaluate error
loss = torch.nn.L1Loss()
MAE_stress = loss(NICE_network.Normalize(pred_stress,prm_s),NICE_network.Normalize(stress_tv,prm_s))
MAE_z = loss(NICE_network.Normalize(pred_svars[:,:,-1:],prm_z),NICE_network.Normalize(svars_tv[:,:,-1:],prm_z))
print("MAE stress : ", MAE_stress,
      "\nMAE z : ", MAE_z)

In [ ]:
fig = plt.figure()
ax = fig.subplots(1)
for i in np.arange(len(ntrainval)):
    ax.plot(strain_t_tv[:,i,1],svars_tv[:,i,1].detach().numpy(),linewidth=5,color='black',alpha=0.2,markersize=0,marker='.')
    ax.plot(strain_t_tv[:,i,1],pred_svars[:,i,3].cpu().detach(),linewidth=2,color=colorb,markersize=0,marker='.')
ax.set_ylabel('$\phi$ (-)')
ax.set_xlabel('$\\varepsilon_s$ (%)')
ax.grid()
plt.show()

fig = plt.figure()
ax = fig.subplots(1)
for i in np.arange(len(ntrainval)):
    ax.plot(strain_t_tv[:,i,1],stress_tv[:,i,1].cpu().detach(),linewidth=5,color='black',alpha=0.2,markersize=0,marker='.')
    ax.plot(strain_t_tv[:,i,1],pred_stress[:,i,1].cpu().detach(),linewidth=2,color=colorb,markersize=0,marker='.')
ax.set_ylabel('$q$ (MPa)')
ax.set_xlabel('$\\varepsilon_s$ (-)')
ax.set_ylim(0,30)
ax.grid()
plt.show()

fig = plt.figure()
ax = fig.subplots(1)
for i in np.arange(len(ntrainval)):
    ax.plot(strain_t_tv[:,i,0],stress_tv[:,i,0].cpu().detach(),linewidth=5,color='black',alpha=0.2,markersize=0,marker='.')
    ax.plot(strain_t_tv[:,i,0],pred_stress[:,i,0].cpu().detach(),linewidth=2,color=colorb,markersize=0,marker='.')
ax.set_ylabel('$p$ (MPa)')
ax.set_xlabel('$\\varepsilon_v$ (-)')
ax.set_ylim(0,20)
ax.grid()
plt.show()

In [ ]:
svars_test.requires_grad=True

t = prm_dt*torch.linspace(0., data_size-1, (data_size)).to(device)
NICE_network.init_interp(dstrain_test,t)
sol = root(NICE_network.find_elastic_strain,
           args=([svars_test[0,:,:1].reshape(-1,1),
                 svars_test[0,:,-1:].reshape(-1,1)],
                 stress_test[0].reshape(-1,2)),
           x0=np.zeros((stress_test.shape[1],2)),
           tol=1e-12)
eps_e_0 = torch.from_numpy(sol.x.reshape(-1,2))
ueps_e_0 = NICE_network.DeNormalize(eps_e_0,NICE_network.prm_ee)
usvars = torch.cat((ueps_e_0,svars_test[0]),-1)
Ntest = np.arange(0,dstrain_test.shape[1])
pred_svars,pred_stress,pred_diss = NICE_network.integrate(dstrain_test,usvars,t,Ntest)

# Evaluate error
loss = torch.nn.L1Loss()
MAE_stress = loss(NICE_network.Normalize(pred_stress,prm_s),NICE_network.Normalize(stress_test,prm_s))
MAE_z = loss(NICE_network.Normalize(pred_svars[:,:,-1:],prm_z),NICE_network.Normalize(svars_test[:,:,-1:],prm_z))
print("MAE stress : ", MAE_stress,
      "\nMAE z : ", MAE_z)

In [ ]:
fig = plt.figure()
ax = fig.subplots(1)
for i in np.arange(len(ntest)):
    ax.plot(strain_t_test[:,i,1],stress_test[:,i,1].cpu().detach(),linewidth=5,color='black',alpha=0.2,markersize=0,marker='.')
    ax.plot(strain_t_test[:,i,1],pred_stress[:,i,1].cpu().detach(),linewidth=2,color=colorb,markersize=0,marker='.')
ax.set_ylabel('$q$ (MPa)')
ax.set_xlabel('$\\varepsilon_s$ (-)')
ax.grid()
ax.set_ylim(0,30)
plt.show()

fig = plt.figure()
ax = fig.subplots(1)
for i in np.arange(len(ntest)):
    ax.plot(strain_t_test[:,i,0],stress_test[:,i,0].cpu().detach(),linewidth=5,color='black',alpha=0.2,markersize=0,marker='.')
    ax.plot(strain_t_test[:,i,0],pred_stress[:,i,0].cpu().detach(),linewidth=2,color=colorb,markersize=0,marker='.')
ax.set_ylabel('$p$ (MPa)')
ax.set_xlabel('$\\varepsilon_v$ (-)')
ax.set_ylim(0,20)
ax.grid()
plt.show()

fig = plt.figure()
ax = fig.subplots(1)
for i in np.arange(len(ntest)):
    ax.plot(strain_t_test[:,i,1],svars_test[:,i,1].detach().numpy(),linewidth=5,color='black',alpha=0.2,markersize=0,marker='.')
    ax.plot(strain_t_test[:,i,1],pred_svars[:,i,3].detach().numpy(),linewidth=2,color=colorb,markersize=0,marker='.')
ax.set_ylabel('$\phi$ (-)')
ax.set_xlabel('$\\varepsilon_s$ (-)')
ax.grid()
plt.show()

fig = plt.figure()
ax = fig.subplots(1)
for i in np.arange(len(ntest)):
    ax.plot(pred_diss[:,i].cpu().detach(),linewidth=3,alpha=0.3,color=colorb,markersize=0,marker='.')
ax.set_ylabel('$d$')
ax.grid()
# ax.set_ylim(0,100)
plt.show()

### 5. Inference

In [ ]:
file = './dataset/benchmark2_inference_UndrainedTriaxial_phi08'
with open(file, 'rb') as f_obj:
    data = pickle.load(f_obj)
[stress_t,strain_t,svars_e_t,epl,r_t,rs_t,stress_tdt,strain_tdt,svars_e_tdt,epl_tdt,r_tdt,rs_tdt,dt,n_reset]= data
batch_time = n_reset
data_size=n_reset

# strain protocol
dstrain = (strain_tdt - strain_t)/prm_dt

# initial condition and data for comparison
strain_t = np.reshape(strain_t,(batch_time,-1,dim),order='F')
dstrain = np.reshape(dstrain,(batch_time,-1,dim),order='F')
r_t = np.reshape(r_t,(batch_time,-1,1),order='F')
rs_t = np.reshape(rs_t,(batch_time,-1,1),order='F')
el_strain_t = np.reshape(svars_e_t,(batch_time,-1,dim),order='F')
stress_t = np.reshape(stress_t,(batch_time,-1,dim),order='F')
data_size = strain_t.shape[0]
number_IC = strain_t.shape[1]

svars = torch.cat((torch.from_numpy(np.float64(r_t)),torch.from_numpy(np.float64(rs_t))),-1).to(device)
stress = torch.from_numpy(np.float64(stress_t)).to(device)
dstrain = torch.from_numpy(np.float64(dstrain)).to(device)

In [ ]:
NICE_network.inference=True
t = torch.arange(0,prm_dt*data_size,prm_dt)
initial_conditions = torch.cat((svars[0,:,:1].reshape(-1,1),svars[0,:,-1:].reshape(-1,1),stress[0].reshape(-1,2)),-1)

In [ ]:
idx = np.arange(0,number_IC)
initial_conditions.requires_grad=True
NICE_network.init_interp(dstrain,t)
sol = root(NICE_network.find_elastic_strain,
           args=([initial_conditions[:,:1],initial_conditions[:,1:2]],initial_conditions[:,2:]),
           x0=np.zeros((number_IC,dim)),
           tol=1e-12)
eps_e_0 = torch.from_numpy(sol.x.reshape(-1,2))
ueps_e_0 = NICE_network.DeNormalize(eps_e_0,NICE_network.prm_ee)
usvars = torch.cat((ueps_e_0,svars[0]),-1)

pred = NICE_network.integrate(dstrain,usvars,t,idx)
pred_svars,pred_stress,pred_diss = pred
pred_svars = pred_svars.cpu().detach().numpy()
pred_stress = pred_stress.cpu().detach().numpy()
pred_diss = pred_diss.cpu().detach().numpy()

In [ ]:
stress_ratio = stress_t[:,:,1]/stress_t[:,:,0]
pred_stress_ratio = pred_stress[:,:,1]/pred_stress[:,:,0]

In [ ]:
nid=1
for idd in range(6):
    fig = plt.figure(dpi=100, figsize=(2.5, 1.5), tight_layout=True)
    axes = fig.subplots(1)
    axes.plot(strain_t[:,idd:idd+nid,1],stress_ratio[:,idd:idd+nid],marker='o',markerfacecolor='white',linestyle='-',
        color='black',alpha=0.2,linewidth=5,markersize=0,label='ref')
    axes.plot(strain_t[::2,idd:idd+nid,1],stress_ratio[::2,idd:idd+nid],marker='o',markerfacecolor='white',markeredgewidth=0.0,linestyle='-',
        color='black',alpha=0.5,linewidth=0,markersize=2.5)
    axes.plot(strain_t[:,idd:idd+nid,1],pred_stress_ratio[:,idd:idd+nid],alpha=1,linewidth=2,color=colorb,
            markersize=0,markeredgewidth=0.0,marker='.')
    axes.set_ylabel('$q/p$ (-)')
    axes.set_xlabel('$\\varepsilon_s$ (%)')
    every_nth=2
    for n, label in enumerate(axes.xaxis.get_ticklabels()):
        if n % every_nth == 0: label.set_visible(False)
    axes.grid()    
    axes.set_ylim(0,2.5)
    plt.show()

In [ ]:
nid=1
for idd in range(6):
    fig = plt.figure(dpi=100, figsize=(2.5, 1.5), tight_layout=True)
    axes = fig.subplots(1)
    axes.plot(strain_t[:,idd:idd+nid,1],rs_t[:,idd:idd+nid,-1],marker='o',markerfacecolor='white',linestyle='-',
        color='black',alpha=0.2,linewidth=5,markersize=0,label='ref')
    axes.plot(strain_t[::2,idd:idd+nid,1],rs_t[::2,idd:idd+nid,-1],marker='o',markerfacecolor='white',markeredgewidth=0.0,linestyle='-',
        color='black',alpha=0.5,linewidth=0,markersize=2.5)
    axes.plot(strain_t[:,idd:idd+nid,1],pred_svars[:,idd:idd+nid,-1],alpha=1,linewidth=2,color=colorb,
            markersize=0,markeredgewidth=0.0,marker='.')
    axes.set_ylabel('$\phi$ (-)')
    axes.set_xlabel('$\\varepsilon_s$ (%)')
    axes.grid()
    every_nth=2
    for n, label in enumerate(axes.xaxis.get_ticklabels()):
        if n % every_nth == 0: label.set_visible(False)
    axes.set_ylim(0.65,0.85)
    plt.show()

### 6. Save model

In [ ]:
PATH = './saved/[state]NICE_benchmark2_dense'
torch.save(NICE_network.state_dict(), PATH)

In [ ]:
with open('./saved/[params]NICE_benchmark2_dense', 'wb') as f_obj:
     pickle.dump(norm_params, f_obj)